In [ ]:
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.io import read_image

In [ ]:

weights = MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1
transforms = weights.transforms()
model = maskrcnn_resnet50_fpn_v2(weights=weights)
model = model.eval();

In [ ]:
from PIL import Image
import numpy as np

input_image = Image.open('exampleimg.jpg')
dp = read_image('exampleimg.jpg')
p_tensor = T.ToTensor()(input_image)
# display(input_image)
np_img = torch.tensor(np.asarray(input_image)).unsqueeze(0)
np_img =  np_img.permute(0, 3, 1, 2)

In [ ]:
input_tensor = transforms(input_image).unsqueeze(0)
input_tensor.shape

In [ ]:
output = model(input_tensor)
output

In [ ]:
score_threshold = 0.75
mask_threshold = 0.5
masks = output[0]['masks'][output[0]['scores'] > score_threshold] > mask_threshold;
pred_labels = [weights.meta["categories"][label] for label in output[0]['labels'][output[0]['scores'] > score_threshold]]
pred_label_unq = [pred_labels[i] + str(pred_labels[:i].count(pred_labels[i]) + 1) for i in range(len(pred_labels))]



masks = masks.squeeze(1)
print(pred_label_unq)

In [ ]:
from torchvision.utils import draw_segmentation_masks
import random

def random_color_gen(n):
    return [tuple(random.randint(0,255) for i in range(3)) for i in range(n)]


print(random_color_gen(len(pred_labels)))
masked_img = draw_segmentation_masks(np_img.squeeze(0), masks, alpha=0.9, colors=random_color_gen(len(pred_labels)))

In [ ]:
masked_img = T.ToPILImage()(masked_img)

In [ ]:
display(masked_img)

In [ ]:
person_mask = masks[8] > 0.5
bg_removed = person_mask*input_tensor
bg_removed_img = T.ToPILImage()(bg_removed.squeeze(0))
display(bg_removed_img)

# Blurring the background

In [ ]:
import math

def get_gaussian_kernel(kernel_size=15, sigma=20, channels=3):
    # Create a x, y coordinate grid of shape (kernel_size, kernel_size, 2)
    x_coord = torch.arange(kernel_size)
    x_grid = x_coord.repeat(kernel_size).view(kernel_size, kernel_size)
    y_grid = x_grid.t()
    xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()

    mean = (kernel_size - 1)/2.
    variance = sigma**2.

    # Calculate the 2-dimensional gaussian kernel which is
    # the product of two gaussian distributions for two different
    # variables (in this case called x and y)
    gaussian_kernel = (1./(2.*math.pi*variance)) *\
                      torch.exp(
                          -torch.sum((xy_grid - mean)**2., dim=-1) /\
                          (2*variance)
                      )

    # Make sure sum of values in gaussian kernel equals 1.
    gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)

    # Reshape to 2d depthwise convolutional weight
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)

    gaussian_filter = nn.Conv2d(in_channels=channels, out_channels=channels,
                                kernel_size=kernel_size, padding='same', groups=channels, bias=False)

    gaussian_filter.weight.data = gaussian_kernel
    gaussian_filter.weight.requires_grad = False
    
    return gaussian_filter

blur = get_gaussian_kernel()
blurred_tensor = blur(input_tensor)
blurred_img = T.ToPILImage()(blurred_tensor.squeeze(0))

display(blurred_img)

In [ ]:
final_img = blurred_tensor
final_img[:, :, person_mask.squeeze(0)] = bg_removed[:, :, person_mask.squeeze(0)];

In [ ]:
final_img = T.ToPILImage()(final_img.squeeze(0))

display(final_img)